In [1]:
#authentication
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Workspace

ia = InteractiveLoginAuthentication(tenant_id='16b3c013-d300-468d-ac64-7eda0820b6d3')

# You can find tenant id under azure active directory->properties
ws = Workspace.get(name='Prod',
                     subscription_id='fe38c376-b42a-4741-9e7c-f5d7c31e5873',
                     resource_group='ProdRG',auth=ia)

## ** Create Tabular Dataset **

In [2]:
#creates an unregistered TabularDataset from a web url.
from azureml.core.dataset import Dataset

web_path ='https://dprepdata.blob.core.windows.net/demo/Titanic.csv'
titanic_ds = Dataset.Tabular.from_delimited_files(path=web_path)

In [ ]:
#creates an unregistered TabularDataset from a data store
from azureml.core import Workspace, Datastore, Dataset

datastore_name = ws.get_default_datastore()
weather_ds = 'weather-data-florida'

#load data to blob storage from local folder (weather-data) to be able to create a dataset
datastore_name.upload('weather-data', weather_ds, overwrite=True, show_progress=True)
    
datastore_path = [(datastore_name, weather_ds + '/*/*/data.parquet')]
dataset        = Dataset.Tabular.from_parquet_files(path=datastore_path, partition_format = weather_ds + '/{partition_time:yyyy/MM}/data.parquet')

#Assign "datetime" column as timestamp and "partition_time" from folder path as partition_timestamp for Tabular Dataset to activate Time Series related APIs. The column to be assigned should be a Date type.
tsd = dataset.with_timestamp_columns(timestamp='datetime', partition_timestamp='partition_time')

# register dataset to Workspace
registered_ds = tsd.register(ws, weather_ds, create_new_version=True, description='Data for Tabular Dataset with time-series trait demo.', tags={ 'type': 'TabularDataset' })

#weather_ds = Dataset.Tabular.from_delimited_files(path=datastore_paths)

### **Example for how to use unregistered dataset in the training script**

In [8]:
'''import os
# Create a folder for the script files
script_folder = 'workshop_examples'
os.makedirs(script_folder, exist_ok=True)

print(script_folder)'''

workshop_examples


In [9]:
'''
%%writefile $script_folder/train_titanic.py
#access data in training script

import argparse
from azureml.core import Dataset, Run

parser = argparse.ArgumentParser()
parser.add_argument("--input-data", type=str)
args = parser.parse_args()

run = Run.get_context()
ws = run.experiment.workspace

# get the input dataset by ID
dataset = Dataset.get_by_id(ws, id=args.input_data)

# load the TabularDataset to pandas DataFrame
df = dataset.to_pandas_dataframe()
'''

Overwriting workshop_examples/train_titanic.py


In [10]:
'''
# define compute target, add the code
# define environment as myenv., add the code
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory=script_folder,
                      script='train_titanic.py',
                      # pass dataset as an input with friendly name 'titanic'
                      arguments=['--input-data', titanic_ds.as_named_input('titanic')],
                      compute_target=compute_target,
                      environment=myenv)
                             
# Submit the run configuration for your training run
run = experiment.submit(src)
run.wait_for_completion(show_output=True)
'''

"\nfrom azureml.core import ScriptRunConfig\n\nsrc = ScriptRunConfig(source_directory=script_folder,\n                      script='train_titanic.py',\n                      # pass dataset as an input with friendly name 'titanic'\n                      arguments=['--input-data', titanic_ds.as_named_input('titanic')],\n                      compute_target=compute_target,\n                      environment=myenv)\n                             \n# Submit the run configuration for your training run\nrun = experiment.submit(src)\nrun.wait_for_completion(show_output=True)\n"

**Create File Dataset **

In [ ]:
from azureml.core.dataset import Dataset

web_paths = [
            'http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz',
            'http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz',
            'http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz',
            'http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz'
            ]

mnist_ds = Dataset.File.from_files(path = web_paths)

**Create datasets from datastores**

## REGISTER DATASETS

In [ ]:
#Register dataset



dataset = Dataset.get_by_name(workspace, name='AutoMLE2EPipeline_Classification_train')
data=dataset.take(100).to_pandas_dataframe()

test = TabularDatasetFactory.register_pandas_dataframe(
    data, target=(datastore, "dataset/"), name="AutoMLE2EPipeline_Classification_test")

** Access registered datasets **

In [ ]:
dataset_name = 'titanic_ds'

# Get a dataset by name
titanic_ds = Dataset.get_by_name(workspace=workspace, name=dataset_name)
#returns the latest version of the dataset